# Module 2.05: The Governance Scoreboard

## What's in this notebook
- Compute executive metrics at business decision grains
- Build three scoreboard views (Executive Leaderboard, Segment Check, Distribution View)
- Apply the Anchor Gate → Rank → Veto selection logic
- Declare the starting Defender

## Key Concept: The Scoreboard Recipe
1. Start with raw CV output (granular rows)
2. Decide the decision grain and decision window
3. Aggregate forecasts and actuals FIRST
4. Recompute ERROR and ABS_ERROR at that decision grain
5. THEN compute governance metrics (wMAPE, Bias, Beat Rate, Jitter)

**Order matters. This is non-negotiable.**

In [1]:
import sys
from pathlib import Path

print("=" * 70)
print("DIAGNOSTIC: Python Environment Setup")
print("=" * 70)

print(f"\n1. Python Interpreter:")
print(f"   sys.executable: {sys.executable}")
is_venv = ".venv" in sys.executable
print(f"   Using Poetry venv: {is_venv} {'✓' if is_venv else '✗ WARNING: Not using Poetry venv!'}")

print(f"\n2. Current Working Directory:")
print(f"   Path.cwd(): {Path.cwd()}")

print(f"\n3. Module and Project Root Resolution:")
MODULE_DIR = Path.cwd()
PROJECT_ROOT = MODULE_DIR.parent.parent
print(f"   MODULE_DIR: {MODULE_DIR}")
print(f"   PROJECT_ROOT: {PROJECT_ROOT}")
print(f"   PROJECT_ROOT exists: {PROJECT_ROOT.exists()} {'✓' if PROJECT_ROOT.exists() else '✗'}")
print(f"   src dir exists: {(PROJECT_ROOT / 'src').exists()} {'✓' if (PROJECT_ROOT / 'src').exists() else '✗'}")

print(f"\n4. sys.path (first 5 entries):")
for i, path in enumerate(sys.path[:5]):
    print(f"   [{i}] {path}")
project_in_path = any(str(PROJECT_ROOT) in path for path in sys.path)
print(f"   PROJECT_ROOT in sys.path: {project_in_path} {'✓' if project_in_path else '✗'}")

print(f"\n5. Attempting to import src module:")
sys.path.insert(0, str(PROJECT_ROOT))
try:
    from src import CacheManager, ArtifactManager, get_notebook_name, find_project_root
    print(f"   ✓ Successfully imported from src")
except ImportError as e:
    print(f"   ✗ Failed to import from src: {e}")

print("\n" + "=" * 70)

DIAGNOSTIC: Python Environment Setup

1. Python Interpreter:
   sys.executable: /Users/jackrodenberg/Desktop/_frameworks/real-world-forecasting-foundations/.venv/bin/python
   Using Poetry venv: True ✓

2. Current Working Directory:
   Path.cwd(): /Users/jackrodenberg/Desktop/_frameworks/real-world-forecasting-foundations/notebooks/module_02_baselines

3. Module and Project Root Resolution:
   MODULE_DIR: /Users/jackrodenberg/Desktop/_frameworks/real-world-forecasting-foundations/notebooks/module_02_baselines
   PROJECT_ROOT: /Users/jackrodenberg/Desktop/_frameworks/real-world-forecasting-foundations
   PROJECT_ROOT exists: True ✓
   src dir exists: True ✓

4. sys.path (first 5 entries):
   [0] /Users/jackrodenberg/Desktop/real-world-forecasting-foundations
   [1] /Users/jackrodenberg/Desktop/_frameworks/real-world-forecasting-foundations/notebooks/module_02_baselines/$PYTHONPATH
   [2] /opt/homebrew/Cellar/python@3.12/3.12.9/Frameworks/Python.framework/Versions/3.12/lib/python312.zip


## Section 1: Setup & Imports

In [2]:
# --- Initialize notebook environment ---
import sys
from pathlib import Path

# --- Data manipulation ---
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import plotly.graph_objects as go

# OS & utilities
import warnings
from typing import Optional, Tuple, List, Dict

# --- Local Imports ---
from src import (
    CacheManager,
    ArtifactManager,
    get_notebook_name,
    find_project_root,
)
from src.metrics import MetricsCalculator, MetricResults

# --- Settings ---
warnings.filterwarnings("ignore")
plt.style.use("seaborn-v0_8-whitegrid")
pd.set_option('display.float_format', '{:.4f}'.format)

# --- Paths ---
DATA_DIR = PROJECT_ROOT / "data"
DATA_DIR.mkdir(exist_ok=True)

# --- Managers ---
NB_NAME = get_notebook_name()
cache = CacheManager(PROJECT_ROOT / ".cache" / NB_NAME)
artifacts = ArtifactManager(PROJECT_ROOT / "artifacts")

# --- Data Contract ---
ID_COL = "unique_id"
TIME_COL = "ds"
TARGET_COL = "y"
CUTOFF_COL = "cutoff"
data_contract_cols = [ID_COL, TIME_COL, TARGET_COL, CUTOFF_COL]

# --- Metrics Configuration ---
ANCHOR_MODEL = "SN52"  # Seasonal Naive - fixed forever
SHORT_HORIZON = (1, 4)    # Weeks 1-4: near-term (store-level decisions)
LONG_HORIZON = (5, 13)    # Weeks 5-13: mid-term (SKU-level decisions)

# Initialize metrics calculator
calculator = MetricsCalculator(
    anchor_model=ANCHOR_MODEL,
    id_col=ID_COL,
    time_col=TIME_COL,
    target_col=TARGET_COL,
    cutoff_col=CUTOFF_COL,
)

print("✓ Setup complete")

✓ Setup complete


## Section 2: Load Data

In [3]:
# Load backtest results
df = artifacts.load("enriched_backtest")

# filter out probabalistic interval columns 
mask = (df['model'].str.contains("-hi")) | (df['model'].str.contains("-lo"))

df = df.loc[~mask]

print(f"✓ Loaded 'enriched_backtest'")
print(f"  Shape: {df.shape[0]:,} × {df.shape[1]}")
print(f"  Columns: {list(df.columns)}")
print(f"  Models: {sorted(df['model'].unique())}")

✓ Loaded 'enriched_backtest' from 02_baselines/
   Shape: 55,491,800 × 8
✓ Loaded 'enriched_backtest'
  Shape: 11,098,360 × 8
  Columns: ['unique_id', 'ds', 'y', 'cutoff', 'model', 'y_pred', 'error', 'abs_error']
  Models: ['AutoETS', 'AutoTheta', 'Croston', 'HW52', 'MA4', 'Naive', 'SN52']


In [13]:
df['dept_id'] = df['unique_id'].str.split('_',expand=True)[0]

df['item_id'] = df['unique_id'].str.rsplit(n=2,pat='_',expand=True)[0]

## Section 3: Compute Metrics

In [14]:
# Compute metrics using the new unified MetricsCalculator
print("📊 Computing Executive Metrics...")
print("=" * 70)

# Step 1: Compute short-horizon metrics (weeks 1-4 at store level)
short_results = calculator.compute_metrics(
    df,
    error_level=["unique_id","ds"], # compute errors at unique_id, ds level 
    metric_level="unique_id", # aggregate the errors to 'unique_id' level
    timesteps=[SHORT_HORIZON],
    metrics=["wmape", "bias", "jitter", "beat_rate","fva"],
)
print(f"✓ Computed short-horizon metrics ({short_results.metric_level.shape[0]} series × models × cutoffs)")

# Step 2: Compute long-horizon metrics (weeks 5-13 at SKU level)
long_results = calculator.compute_metrics(
    df,
    error_level="item_id",
    metric_level="item_id",
    timesteps=[LONG_HORIZON],
    metrics=["wmape", "bias", "jitter", "beat_rate","fva"],
)
print(f"✓ Computed long-horizon metrics ({long_results.metric_level.shape[0]} items × models × cutoffs)")

print("=" * 70)

📊 Computing Executive Metrics...
✓ Computed short-horizon metrics (213430 series × models × cutoffs)
✓ Computed long-horizon metrics (21343 items × models × cutoffs)


In [15]:
short_results.metric_level

,unique_id,model,jitter,fva,wmape,bias,beat_rate
0,FOODS_1_001_CA_1,AutoETS,0.5462,0.1446,0.3486,0.0536,0.7500
1,FOODS_1_001_CA_1,AutoTheta,0.5664,0.1505,0.3427,0.0485,0.7500
2,FOODS_1_001_CA_1,Croston,0.4295,0.1316,0.3615,0.0215,0.7500
3,FOODS_1_001_CA_1,HW52,0.2197,0.1286,0.3646,0.1647,0.5000
4,FOODS_1_001_CA_1,MA4,0.6593,0.0822,0.4110,0.1644,0.5000
...,...,...,...,...,...,...,...
213425,HOUSEHOLD_2_516_WI_3,Croston,NaN,-0.0037,0.5421,0.0782,0.5000
213426,HOUSEHOLD_2_516_WI_3,HW52,NaN,-0.0436,0.5820,-0.1519,0.2500
213427,HOUSEHOLD_2_516_WI_3,MA4,NaN,-0.2308,0.7692,0.3077,0.2500
213428,HOUSEHOLD_2_516_WI_3,Naive,0.9623,-0.3077,0.8462,0.8462,0.2500


In [7]:
metric_df = calculator._aggregate_to_metric_level(df=df,metric_level='unique_id')
metric_level_df = calculator._compute_aggregated_errors(agg_df = metric_df)

metric_level = 'unique_id'

metrics = ["wmape", "bias", "jitter", "beat_rate"]

if "wmape" in metrics:
    metric_level_df = calculator._compute_wmape(metric_level_df)

if "bias" in metrics:
    metric_level_df = calculator._compute_bias(metric_level_df)

if "beat_rate" in metrics:
    metric_level_df = calculator._compute_beat_rate(metric_level_df, metric_level)

if "jitter" in metrics:
    metric_level_df = calculator._compute_jitter(metric_level_df, metric_level)

In [8]:
 # Step 4: Remove cutoff from final metric_level_df to ensure single summary per groupbykeys
# (metric_level, model are the groupby keys; cutoff is only used for internal computation)
metric_level_df = calculator._finalize_metric_level(metric_level_df, metric_level)

In [9]:
calculator._aggregate_to_portfolio()

TypeError: MetricsCalculator._aggregate_to_portfolio() missing 2 required positional arguments: 'metric_level_df' and 'metrics'

## Section 4: View 1 - Executive Leaderboard

In [ ]:
# Combine short and long-term into single executive view
short_portfolio = short_results.portfolio.copy()
short_portfolio["horizon_group"] = "1-4 weeks"

long_portfolio = long_results.portfolio.copy()
long_portfolio["horizon_group"] = "5-13 weeks"

executive_leaderboard = pd.concat([
    short_portfolio.reset_index(drop=True),
    long_portfolio.reset_index(drop=True)
], ignore_index=True)

# Format for presentation
executive_leaderboard_display = (
    executive_leaderboard
    [["model", "horizon_group", "wmape", "bias", "beat_rate", "jitter"]]
    .sort_values(["horizon_group", "wmape"])
    .round({"wmape": 4, "bias": 4, "beat_rate": 2, "jitter": 4})
)

print("=" * 80)
print("EXECUTIVE LEADERBOARD (vs Anchor: Seasonal Naive)")
print("=" * 80)
print(executive_leaderboard_display.to_string(index=False))
print("=" * 80)

EXECUTIVE LEADERBOARD (vs Anchor: Seasonal Naive)
    model horizon_group  wmape    bias  beat_rate  jitter
      MA4     1-4 weeks 0.0278 -0.0191    91.6700  0.0249
  AutoETS     1-4 weeks 0.0366 -0.0317    83.3300  0.0277
AutoTheta     1-4 weeks 0.0373 -0.0319    83.3300  0.0278
  Croston     1-4 weeks 0.0479 -0.0441    91.6700  0.0413
     HW52     1-4 weeks 0.0562 -0.0545    91.6700  0.0379
    Naive     1-4 weeks 0.0769 -0.0769    75.0000  0.0490
     SN52     1-4 weeks 0.0973 -0.0971     0.0000  0.0544
  Croston    5-13 weeks 0.2566 -0.0324    57.1000  7.5406
AutoTheta    5-13 weeks 0.2578 -0.0129    56.6600  6.3207
  AutoETS    5-13 weeks 0.2585 -0.0128    56.2600  6.2267
      MA4    5-13 weeks 0.2630 -0.0035    55.1600  6.3106
    Naive    5-13 weeks 0.2966 -0.0620    47.5100  5.8432
     SN52    5-13 weeks 0.2970 -0.0904     0.0000  4.5632
     HW52    5-13 weeks 0.3038 -0.0602    50.6300  4.2024


## Section 5: Visualization - Long-Horizon Distribution

In [55]:
# Create visualization: Long-horizon metrics vs anchor
print(f"\n📈 Long-Horizon Metrics vs Anchor ({ANCHOR_MODEL})")
fig = calculator.boxplot_against_anchor(
    long_results.metric_level,
    metric="wmape",
    clip_quantiles=(0.05, .975),
    show_points=True,
    show_stats=True,
    color_scheme="red_blue",
    figsize=(1000, 700),
    title="Mid-Term WMAPE Distribution vs Seasonal Naive"
)
fig.show()


📈 Long-Horizon Metrics vs Anchor (SN52)

SUMMARY: WMAPE vs SN52
Model               Median      Delta    Delta %        N
------------------------------------------------------------
AutoETS             0.1645   -0.0478     -22.5%    12121
AutoTheta           0.1655   -0.0468     -22.1%    12121
MA4                 0.1719   -0.0404     -19.1%    12121
Croston             0.1784   -0.0339     -16.0%    12121
SN52                0.2123    0.0000       0.0%    12121
HW52                0.2243 +   0.0119 +      5.6%    12121
Naive               0.2324 +   0.0201 +      9.5%    12121



## Section 6: View 2 - Segment Check

Portfolio averages hide landmines. We slice the same metrics by business dimensions.

In [ ]:
# Compute segment-level metrics directly from raw data
print("\n" + "=" * 60)
print("SEGMENT ANALYSIS")
print("=" * 60)

# Check which segment columns are available in the raw data
available_segments = [col for col in ["dept_id", "state_id", "cat_id"] if col in df.columns]

if available_segments:
    for segment_col in available_segments:
        # Compute metrics at segment level directly
        segment_results = calculator.compute_metrics(
            df,
            error_level="item_id",
            metric_level=segment_col,
            timesteps=[LONG_HORIZON],
            metrics=["wmape", "bias", "beat_rate"],
        )

        print(f"\n{'='*60}")
        print(f"SEGMENT: {segment_col}")
        print('='*60)

        # Pivot: models as rows, segments as columns
        pivot = segment_results.metric_level.pivot_table(
            index="model",
            columns=segment_col,
            values="wmape",
            aggfunc="first"
        ).round(4)
        print(pivot)
else:
    print("⚠️  Segment columns (dept_id, state_id, cat_id) not found in data.")
    print("   Skipping segment analysis. Add metadata to enable this view.")

## Section 7: View 3 - Distribution View (Risk Lens)

In [34]:
def compute_improvement_distribution(
    metric_level_df: pd.DataFrame,
    anchor_model: str = ANCHOR_MODEL,
    metric_col: str = "wmape"
) -> pd.DataFrame:
    """
    Compute % improvement vs Anchor for each SKU.
    % improvement = (anchor_wmape - model_wmape) / anchor_wmape * 100
    Positive = model is better than anchor
    """
    # Get anchor performance per SKU (average across cutoffs)
    anchor_by_sku = (
        metric_level_df
        .query(f"model == '{anchor_model}'")
        .groupby("item_id")
        [metric_col]
        .mean()
        .reset_index()
        .rename(columns={metric_col: "anchor_wmape"})
    )
    
    # Get model performance per SKU (average across cutoffs)
    model_by_sku = (
        metric_level_df
        .groupby(["model", "item_id"])
        [metric_col]
        .mean()
        .reset_index()
        .rename(columns={metric_col: "model_wmape"})
    )
    
    # Merge and compute improvement
    improvement = model_by_sku.merge(anchor_by_sku, on="item_id", how="left")
    improvement = improvement.query("anchor_wmape > 0.01").copy()
    
    improvement["pct_improvement"] = (
        (improvement["anchor_wmape"] - improvement["model_wmape"]) 
        / improvement["anchor_wmape"] * 100
    )
    
    return improvement

# Compute improvement distribution
improvement_dist = compute_improvement_distribution(long_results.metric_level)
print(f"✓ Computed improvement distribution: {len(improvement_dist):,} model × SKU combinations")

# Summary statistics
improvement_summary = (
    improvement_dist
    .query(f"model != '{ANCHOR_MODEL}'")
    .groupby("model")
    ["pct_improvement"]
    .agg(["mean", "median", "std", lambda x: (x > 0).mean() * 100])
    .rename(columns={"<lambda_0>": "beat_rate_pct"})
    .round(2)
)

print("\nImprovement Distribution Summary:")
print(improvement_summary)

print("""
Interpretation Guide:
- Case 1 (Safe Default): Tight std, positive median, high beat rate
- Case 2 (Lucky Model): Wide std, near-zero median, moderate beat rate
- Case 3 (Specialist): Low beat rate, but meaningful positive tail
""")

✓ Computed improvement distribution: 21,343 model × SKU combinations

Improvement Distribution Summary:
               mean  median       std  beat_rate_pct
model                                               
AutoETS   -193.9600 18.2600 3411.6200        61.7300
AutoTheta -188.9700 18.7100 3415.4600        62.2500
Croston   -178.6000 15.2800 3399.6500        62.5800
HW52       -85.7500  1.1600 1202.4700        50.9300
MA4       -202.4500 13.8100 3420.6000        58.7100
Naive     -224.6200 -6.6200 3746.4300        46.3100

Interpretation Guide:
- Case 1 (Safe Default): Tight std, positive median, high beat rate
- Case 2 (Lucky Model): Wide std, near-zero median, moderate beat rate
- Case 3 (Specialist): Low beat rate, but meaningful positive tail



In [36]:
long_results.portfolio

,model,wmape,bias,jitter,beat_rate
0,AutoETS,0.2585,-0.0128,6.2267,56.2577
1,AutoTheta,0.2578,-0.0129,6.3207,56.6620
2,Croston,0.2566,-0.0324,7.5406,57.0992
3,HW52,0.3038,-0.0602,4.2024,50.6311
4,MA4,0.2630,-0.0035,6.3106,55.1605
5,Naive,0.2966,-0.0620,5.8432,47.5126
6,SN52,0.2970,-0.0904,4.5632,0.0000


## Section 8: Defender Selection Logic

In [38]:
def select_defender(
    portfolio_df: pd.DataFrame,
    anchor_model: str = ANCHOR_MODEL,
    beat_rate_threshold: float = 50.0,
    bias_threshold: float = 10.0,
    jitter_threshold: float = 0.3
) -> pd.DataFrame:
    """
    Apply Gate → Rank → Veto logic to select the Defender.
    
    1. **Anchor Gate:** Must beat Seasonal Naive on portfolio wMAPE AND beat rate > 50%
    2. **Rank:** Among eligible models, rank by portfolio wMAPE (lower is better)
    3. **Veto Checks:**
       - Beat Rate ≥ threshold
       - Bias within threshold
       - Jitter below threshold
    """
    df = portfolio_df.copy()
    
    # Get anchor performance
    anchor_wmape = df.loc[df["model"] == anchor_model, "wmape"].values[0]
    
    # Step 1: Anchor Gate
    df["passes_anchor_gate"] = (
        (df["wmape"] < anchor_wmape) & 
        (df["beat_rate"] > 50)
    )
    
    # Step 2: Rank by wMAPE
    df["wmape_rank"] = df["wmape"].rank(method="min").astype(int)
    
    # Step 3: Veto checks
    df["beat_rate_pass"] = df["beat_rate"] >= beat_rate_threshold
    df["bias_pass"] = df["bias"].abs() <= bias_threshold
    df["jitter_pass"] = df["jitter"] <= jitter_threshold
    
    # Combined eligibility
    df["all_vetos_pass"] = df["beat_rate_pass"] & df["bias_pass"] & df["jitter_pass"]
    df["eligible_defender"] = df["passes_anchor_gate"] & df["all_vetos_pass"]
    
    # Decision
    df["final_decision"] = "REJECT"
    df["decision_reason"] = ""
    
    for idx, row in df.iterrows():
        reasons = []
        if not row["passes_anchor_gate"]:
            reasons.append("Failed Anchor Gate")
        if not row["beat_rate_pass"]:
            reasons.append(f"Beat Rate {row['beat_rate']:.1f}% < {beat_rate_threshold}%")
        if not row["bias_pass"]:
            reasons.append(f"Bias {row['bias']:+.1f}% exceeds ±{bias_threshold}%")
        if not row["jitter_pass"]:
            reasons.append(f"Jitter {row['jitter']:.3f} > {jitter_threshold}")
        
        df.at[idx, "decision_reason"] = "; ".join(reasons) if reasons else "Eligible"
    
    # Find the Defender: lowest wMAPE among eligible
    eligible = df.query("eligible_defender == True").sort_values("wmape")
    if len(eligible) > 0:
        defender_idx = eligible.index[0]
        df.at[defender_idx, "final_decision"] = "DEFENDER"
        df.at[defender_idx, "decision_reason"] = "Selected as Defender (lowest wMAPE among eligible)"
    
    return df

# Run selection for long-term (mid-term) horizon
defender_decision = select_defender(
    long_results.portfolio,
    anchor_model=ANCHOR_MODEL,
    beat_rate_threshold=50.0,
    bias_threshold=10.0,
    jitter_threshold=10
)

print("\n" + "=" * 80)
print("DEFENDER SELECTION (Mid-Term Horizon)")
print("=" * 80)
print(defender_decision[[
    "model", "wmape", "wmape_rank", "passes_anchor_gate", 
    "beat_rate_pass", "bias_pass", "jitter_pass", 
    "final_decision", "decision_reason"
]].round({"wmape": 4}).to_string(index=False))
print("=" * 80)


DEFENDER SELECTION (Mid-Term Horizon)
    model  wmape  wmape_rank  passes_anchor_gate  beat_rate_pass  bias_pass  jitter_pass final_decision                                    decision_reason
  AutoETS 0.2585           3                True            True       True         True         REJECT                                           Eligible
AutoTheta 0.2578           2                True            True       True         True         REJECT                                           Eligible
  Croston 0.2566           1                True            True       True         True       DEFENDER Selected as Defender (lowest wMAPE among eligible)
     HW52 0.3038           7               False            True       True         True         REJECT                                 Failed Anchor Gate
      MA4 0.2630           4                True            True       True         True         REJECT                                           Eligible
    Naive 0.2966           5   

In [45]:
# Extract the winner
defender_row = defender_decision.query("final_decision == 'DEFENDER'")

if len(defender_row) > 0:
    DEFENDER_MODEL = defender_row["model"].values[0]
    print(f"\n{'='*80}")
    print(f"DEFENDER SELECTED: {DEFENDER_MODEL}")
    print(f"{'='*80}")
    print(f"Mid-term wMAPE: {defender_row['wmape'].values[0]:.4f}")
    print(f"Beat Rate: {defender_row['beat_rate'].values[0]:.1f}%")
    print(f"Bias: {defender_row['bias'].values[0]:+.4f}")
    print(f"Jitter: {defender_row['jitter'].values[0]:.4f}")
    print(f"{'='*80}")
else:
    DEFENDER_MODEL = ANCHOR_MODEL
    print(f"\n⚠️ No model passed all gates. Anchor ({ANCHOR_MODEL}) remains the Defender.")


DEFENDER SELECTED: Croston
Mid-term wMAPE: 0.2566
Beat Rate: 57.1%
Bias: -0.0324
Jitter: 7.5406


## Section 9: Exports

In [ ]:
# Create output directory
OUTPUT_DIR = DATA_DIR / "output"
OUTPUT_DIR.mkdir(exist_ok=True)

# Export 1: Executive leaderboard
scoreboard_export = executive_leaderboard[[
    "model", "horizon_group", "wmape", "bias", "beat_rate", "jitter"
]].copy()
scoreboard_export.to_parquet(OUTPUT_DIR / "scoreboard_summary.parquet", index=False)
print(f"✓ Exported: scoreboard_summary.parquet ({len(scoreboard_export)} rows)")

# Export 2: SKU-level metrics for diagnostics
sku_level_export = long_results.metric_level[[
    "model", "item_id", "cutoff", "sum_forecast", "sum_actual",
    "error", "abs_error", "wmape", "bias", "beat_indicator"
]].copy()
sku_level_export.to_parquet(OUTPUT_DIR / "scoreboard_by_sku.parquet", index=False)
print(f"✓ Exported: scoreboard_by_sku.parquet ({len(sku_level_export):,} rows)")

# Export 3: Defender decision audit trail
decision_export = defender_decision[[
    "model", "wmape", "bias", "beat_rate", "jitter",
    "passes_anchor_gate", "wmape_rank", "beat_rate_pass", "bias_pass", 
    "jitter_pass", "final_decision", "decision_reason"
]].copy()
decision_export.to_parquet(OUTPUT_DIR / "defender_decision_table.parquet", index=False)
print(f"✓ Exported: defender_decision_table.parquet ({len(decision_export)} rows)")

print("\n" + "="*80)
print("NOTEBOOK COMPLETE")
print("="*80)
print(f"""
Artifacts created:
- scoreboard_summary.parquet (Executive Leaderboard)
- scoreboard_by_sku.parquet (SKU-level for diagnostics)
- defender_decision_table.parquet (Selection audit trail)

Key Decisions:
- Anchor (Fixed Forever): {ANCHOR_MODEL}
- Defender (Production Standard): {DEFENDER_MODEL}

Next Steps:
- Module 2.6: Diagnostics — Where does the Defender fail?
- Module 2.7: Detailed Forensics — Why does it fail?
""")